# 워드 임베딩

### 벡터화

- 기계가 자연어 처리를 원활히 할 수 있도록, 전처리 과정에서 텍스트를 벡터로 변환하는 과정

#### 1) BoW / DTM(Document-Term Matrix, 문서 단어 행렬)
- BoW는 단어의 순서를 고려하지 않고, 단어의 등장 빈도만을 고려해 단어를 벡터화하는 방법이다.
- 한계
    - (1) Sparsity
    - (2) Frequent words has more power
    - (3) Ignoring orders
    - (4) OOV = 단어 사전에 없는 단어를 처리할 수 없는 문제

#### 2) TF/IDF(Term Frequence * Inverse Document Frequency)

- TF: 한 문서 내에서 등장 빈도에 따라 단어의 중요도를 score로 나타낸 것
- 즉 문장을 구성하는 단어들의 원핫 벡터들을 모두 더해서 문장의 단어 개수로 나눈 것이다.
- IDF: 의미는 중요하지 않은데 단지 모든 문서에서 자주 등장하는 단어의 중요도를 score로 계산한 것(log를 씌워서 모든 문서에서 해당 단어가 등장할수록 값이 작아짐)

#### 3) 벡터화한 걸 어디에 쓰나?
1. 유사도 계산 -> DTM(문서를 행, 단어를 열로 구성
2. 머신러닝 모델의 입력 값으로 사용

<hr>

## 실습 1: 원핫 인코딩 구현

In [1]:
!pip install konlpy

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import re
from konlpy.tag import Okt
from collections import Counter

In [3]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

In [4]:
# 전처리
reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")
text = reg.sub('', text)
print(text)

임금님 귀는 당나귀 귀 임금님 귀는 당나귀 귀 실컷 소리치고 나니 속이 확 뚫려 살 것 같았어


In [5]:
# 토큰화
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


In [6]:
# 단어장 만들기
# 빈도수가 높은 순서대로 배치
vocab = Counter(tokens)
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [7]:
# 단어 등장 빈도 확인
vocab['임금님']

2

In [8]:
# most_common()은 상위 빈도수의 단어를 주어진 수만큼 리턴
# 상위 빈도수 5개 단어 출력
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


In [9]:
# 상위 5개 단어만으로 사전을 만들기
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


In [10]:
# 원핫벡터 만들기
def one_hot_encoding(word, word2index):
    one_hot_vector = [0]*(len(word2index))
    index = word2index[word]
    one_hot_vector[index-1] = 1
    return one_hot_vector

In [11]:
one_hot_encoding("임금님", word2idx)

[0, 1, 0, 0, 0]

In [12]:
# keras로 원핫벡터 만들기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [13]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

[['강아지', '고양이', '강아지'], ['애교', '고양이'], ['컴퓨터', '노트북']]

In [14]:
# 단어 사전 만들기
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


In [15]:
# padding 토큰을 고려해 1을 더해줌
vocab_size = len(t.word_index) + 1

In [16]:
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


In [17]:
one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


<hr>

## 워드 임베딩

### - 희소 벡터의 문제점

- DTM, TF-IDF, 원핫벡터는 단어장의 크기게 영향을 받는 희소 벡터이다. 이들은 전체 차원 중 하나의 원소만 1이고 나머지는 모두 0인 값을 가진다.
- 이러한 희소 벡터에는 차원의 저주 문제가 있다. 정보 밀도가 작아지는 것, 즉 차원이 커지는 것과 머신러닝 모델의 성능에는 어떤 연관 관계가 있다.
- 데이터에서 모델을 학습할 때 독립적 샘플이 많을수록 학습이 잘 되는 반면 차원이 커질수록 학습이 어려워지고 더 많은 데이터를 필요로 한다. 이는 선형 회귀 문제에서 feature가 많을 때 모델의 일반화 성능이 잘 나오지 않는 것과도 같은 맥락이다.

![image](https://user-images.githubusercontent.com/80008411/135985433-e3aa6f26-8871-4b5d-81e3-91fdf71c3786.png)

![image](https://user-images.githubusercontent.com/80008411/135986177-825f3ea6-6d84-4d16-b316-d26123ecb64d.png)

![image](https://user-images.githubusercontent.com/80008411/135986350-5dd9d3d8-b166-4ede-a41e-3cb923b5b2e6.png)

- 또 다른 문제는 의미상 유사도를 계산할 수 없다는 것이다.
- 따라서 그 대안으로 워드 임베딩이 제안되었다.

### - 워드 임베딩

- 워드 임베딩은 기계가 단어장 크기보다 적은 차원의 밀집 벡터를 학습하는 방식이다. 이를 통해 얻는 밀집 벡터는 각 차원이 0과 1이 아닌 다양한 실수 값을 가진다.
- 밀집 표현(Dense Representation)은 벡터의 차원을 단어 집합의 크기로 상정하지 않고, 사용자가 설정한 값으로 모든 단어의 벡터 표현의 차원을 맞춘다.
- 워드 임베딩 방법론으로는 LSA, Word2Vec, FastText, Glove 등이 있다.
- keras의 Embedding()에서는 위 방법들을 사용하지는 않지만, 단어를 랜덤한 값을 가지는 밀집 벡터로 변환한 뒤에, 인공 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습한다.

<hr>

## 워드 임베딩 1: Word2Vec

- 학습된 한국어 Word2Vec 벡터들로 연산한 결과를 제공하는 [사이트](https://word2vec.kr/search/)
- 예를 들어, 박찬호 - 야구 + 축구 = 호나우두
- 함께 사용되는 단어들(neighbor)일수록 유사도가 높다고 판단
- 크게 CBoW와 skip-gram의 두 가지 방법이 있다.
- CBoW는 주변에 있는 단어들을 통해 중간 단어를 예측하는 방법이고, skipgram은 반대로 중간에 있는 단어로 주변 단어들을 예측하는 방법이다.
- skipgram에서 window size=1일 경우, 해당 단어 옆의 1개 단어만 살펴보겠다는 의미

![image](https://user-images.githubusercontent.com/80008411/136125368-07bc7fa0-42e5-4c7d-9003-417fea5872cf.png)

- 아래 그림에서 hidden layer가 바로 word2vec이 된다.
- output에 softmax를 취하므로 모두 합한 값은 1이 된다(확률).
- 마지막으로 output과 target의 차이로 cross-entropy를 계산해서 back-propagation을 해서 가중치를 업데이트
- 이 가중치가 embedding 값이다.

![image](https://user-images.githubusercontent.com/80008411/136125634-6b6f69db-17ba-4785-94db-c9588c408582.png)

- input은 그냥 단어를 원핫인코딩한 벡터
- 우리가 구하고 싶은 embedding은 hidden layer, 즉 w 가중치
- 결국 이 embedding 벡터를 가지고 유사도를 따질 수 있다. 그리고 심지어 그래프에 plot하면 물리적 거리가 당연히 가깝게 나옴

![image](https://user-images.githubusercontent.com/80008411/136125912-a5a01145-a4e4-4815-88da-c98772bafdd9.png)

![image](https://user-images.githubusercontent.com/80008411/136125992-6f65049d-bffd-4b5e-86d4-a47600ee60f2.png)

![image](https://user-images.githubusercontent.com/80008411/136126021-556f95fe-e649-4970-9d69-18c265a5bba5.png)


#### - 분포 가설(Distributional Hypothesis)

- 위와 같은 Word2Vec의 방식은 바로 분포 가설을 따른다. 즉 어떤 단어들의 의미를 보려면 주변 단어들을 보면 알 수 있다는 것이다. 비슷한 문맥에서 같이 등장하는 경향이 있는 단어들은 비슷한 의미를 가진다는 것이다.

### Word2Vec (1) CBoW(Continous Bag of Words)

- 주변 단어(context word)들로 중심 단어(center word)를 예측하는 방법
- 중심 단어를 예측하기 위해 앞, 뒤로 몇 개의 단어를 볼지 그 범위를 윈도우(window)라고 한다.
- 윈도우 크기를 정하면, 이 윈도우를 움직이면서 주변 단어와 중심 단어를 바꿔가며 학습을 위한 데이터 셋을 만들 수 있다. 이를 슬라이딩 윈도우라고 한다.

![image](https://user-images.githubusercontent.com/80008411/136127281-1865ae7c-b501-448b-840a-82c39947ad86.png)

![image](https://user-images.githubusercontent.com/80008411/136128473-2c432ee8-978c-4c97-add0-ad51c4f54e90.png)

- 위 그림은 원핫 벡터로 변환된 다수의 주변 단어를 이용해 원핫 벡터로 변환된 중심 단어를 예측할 때의 CBoW의 동작 메커니즘을 보여준다.
- 윈도우 크기가 m이라면 2m개의 주변 단어를 이용해 1개의 중심 단어를 예측하는 과정에서 두 개의 가중치 행렬을 학습하는 것이 목적이다.

- 주황색 행렬이 첫 번째 가중치 행렬 W, 초록색이 두 번째 가중치 행렬 W'이 된다.
- 이는 인공 신경망 구조상 입력층, 은닉층, 출력층의 3개 층으로 구성된 것으로 볼 수 있다.
- 입력층과 출력층의 크기는 단어 집합의 크기인 V로 고정된다.
- 그러나 은닉층의 크기는 사용자가 지정하는 하이퍼파라미터이며, N, 즉 word embedding dimension을 의미한다.

![image](https://user-images.githubusercontent.com/80008411/136139077-c5f3a488-9aca-45f9-8272-e6e627764d4f.png)

- 주변 단어로 선택된 각각의 원핫 벡터는 첫 번째 가중치 행렬과 곱해지게 되고, 이 가중치 행렬의 크기는 V x N이다.
- 이때 원핫 벡터는 각 단어의 해당하는 인덱스 i에서만 1의 값을 가지므로, 가중치 행렬과의 곱은 행렬의 i번째 row를 그대로 가져오는 것과 동일하다.
- 이처럼 마치 테이블에서 값을 그대로 룩업해오는 것과 같다고 하여 lookup table이라고 부른다.
- 룩업 테이블을 거쳐 얻은 2m개의 주변 단어 벡터들은 각각 N의 크기를 가진다. CBoW에서는 이 벡터들을 모두 합한 값이나 평균 값을 은닉층의 최종 결과로 도출한다.
- Word2Vec에서는 은닉층에서 활성화 함수나 편향을 더하는 연산은 하지 않는다. 때문에 다른 신경망의 은닉층과 구분지어 투사층(Projection layer)라고도 한다.

![image](https://user-images.githubusercontent.com/80008411/136141829-d27c2e7a-b2c0-4fb5-8a59-ad317ef1337a.png)

- 은닉층에서 생성된 N차원 벡터는 두 번째 가중치 행렬과 곱해진다. 이 행렬의 크기는 N x V이고, 따라서 곱셈의 결과로 V 차원의 벡터를 얻을 수 있다.
- 출력층은 활성화 함수로 소프트맥스 함수를 사용하므로 이 V차원의 벡터는 결국 모든 차원의 총합이 1이 되는 벡터로 변환된다.
- 이렇게 얻은 출력층의 벡터를 중심 단어의 원핫 벡터와 비고해 그 손실이 최소화되도록 학습한다.
- 학습이 완료되면 N차원 크기를 갖는 W의 row나 W'의 column 중 어떤 것을 임베딩 벡터로 사용할지 결정한다. 또는 둘의 평균 값을 선택하기도 한다.

### Word2Vec (2) Skip-gramp과 Negative Sampling

- skipgram은 CBoW와 반대로 중심 단어로 주변 단어를 예측한다. 따라서 데이터셋의 형식은 (중심 단어, 주변 단어)로 다음과 같다.
- (i, like) (like, I), (like, natural), (natural, like), (natural, language), (language, natural), (language, processing), (processing, language)

![image](https://user-images.githubusercontent.com/80008411/136142187-d5abd32f-d9da-4cfe-b369-ad23fd7ce2dc.png)

- CBoW의 메커니즘과 다른 점은 중심 단어로부터 주변 단어를 예측하는 점과 은닉층에서 벡터의 덧셈과 평균 계산 과정이 없다는 것이다.
- 또 CBoW와 마찬가지로 학습 후에 가중치 행렬 W의 row 또는 W'의 column으로부터 임베딩 벡터를 얻을 수 있다.

<br>

- 대체로 Word2Vec을 사용할 때는 SGNS(Skip-Gram with Negative Sampling)을 사용한다. 즉 skipgram을 사용하면서 네거티브 샘플링도 사용하는 것. 왜냐하면 word2vec의 구조는 연산량이 지나치게 많아서 실제로 사용하기 어렵다!
- 예를 들어 중심 단어와 주변 단어가 사과, 딸기와 같은 과일이라면, 필통이나 연필 같이 연관 관계가 없는 단어들의 임베딩 값을 매번 업데이트할 필요가 없다. 따라서 네거티브 샘플링은 연산량을 줄이기 위해 소프트맥스 함수를 사용한 V개 중 1개를 고르는 다중 클래스 분류 문제를 시그모이드 함수를 사용한 이진 분류 문제로 바꾸는 것이다.

![image](https://user-images.githubusercontent.com/80008411/136146463-d2ffc51d-369b-4aa9-a0c1-566934f2f93f.png)

![image](https://user-images.githubusercontent.com/80008411/136146484-fd9c53df-4877-4d6a-8a04-5712be47c427.png)

![image](https://user-images.githubusercontent.com/80008411/136146502-192e3fcd-0185-4cce-bdc1-05efec7a2a4a.png)

- target word의 경우 target label을 1로, 다른 단어들은 0으로 labeling해주는 것! 이때 거짓(negative) 데이터셋을 만들기 때문에 이를 네거티브 샘플링이라고 한다.

![image](https://user-images.githubusercontent.com/80008411/136147026-8cd9f62f-7ef9-449c-9322-84501b0d2a42.png)

- 이렇게 완성된 데이터셋으로 학습하면 이진 분류 문제로 간주할 수 있다. 중심 단어와 주변 단어를 내적하고, 출력층의 시그모이드 함수를 지나게 하여 1 또는 0의 레이블로부터 오차를 구해서 역전파를 수행한다.
- 이렇게 하면 기존 소프트맥스 함수 방식보다 연산량이 획기적으로 줄어든다.

### Word2Vec (3) 실습

- gensim 패키지 이용해서 토픽 모델링, 훈련 데이터는 NLTK에서 제공하는 corpus 이용


In [1]:
!pip install nltk
!pip install gensim

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/abc.zip.
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# corpus 불러오기
from nltk.corpus import abc

corpus = abc.sents()

In [4]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [5]:
print('코퍼스의 크기 :',len(corpus))

코퍼스의 크기 : 29059


In [6]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=5, workers=4, sg=0)

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


- vector size = 학습 후 임베딩 벡터의 차원
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도수 제한(빈도가 적은 단어들은 학습하지 않음)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBoW, 1은 Skip-gram.

In [7]:
model_result = model.wv.most_similar('man')
print(model_result)

[('woman', 0.9233418107032776), ('skull', 0.911030113697052), ('Bang', 0.905648946762085), ('asteroid', 0.9052114486694336), ('third', 0.9020071625709534), ('baby', 0.8994219303131104), ('dog', 0.898607611656189), ('bought', 0.8975202441215515), ('rally', 0.8912495374679565), ('disc', 0.8889137506484985)]


In [8]:
# 모델 저장하기
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('~/aiffel/aiffel_projects/goingdeeper/GD5_word_embedding/w2v')
loaded_model = KeyedVectors.load_word2vec_format('~/aiffel/aiffel_projects/goingdeeper/GD5_word_embedding/w2v')

In [9]:
# 모델 로드가 잘 되었는지 확인
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.9233418107032776), ('skull', 0.911030113697052), ('Bang', 0.905648946762085), ('asteroid', 0.9052114486694336), ('third', 0.9020071625709534), ('baby', 0.8994219303131104), ('dog', 0.898607611656189), ('bought', 0.8975202441215515), ('rally', 0.8912495374679565), ('disc', 0.8889137506484985)]


#### - Word2Vec의 OOV 문제

- 사전에 없는 단어에 대해 임베딩 벡터값을 얻을 수 없다.
- 없는 단어를 모델에 입력하면 에러가 난다.
> KeyError: "Key 'overacting' not present"

In [10]:
# 에러가 나더라도 놀라지 마세요.
loaded_model.most_similar('overacting')

KeyError: "Key 'overacting' not present"

<hr>

## 임베딩 벡터의 시각화

- 구글이 공개한 임베딩 벡터의 시각화 오픈소스인 임베딩 프로젝터 사용
- 이를 통해 어떤 임베딩 벡터들이 가까운 거리에 군집되어 있고, 특정 임베딩 벡터와 유클리드 거리나 코사인 유사도가 높은지 확인할 수 있다.
- 이를 위해서는 이미 저장된 모델로부터 벡터 값이 저장된 파일과 메타파일을 불러와야 한다!
> $ python -m gensim.scripts.word2vec2tensor --input ~/aiffel/word_embedding/w2v --output ~/aiffel/word_embedding/w2v

- 위 커맨드를 실행하면 w2v_metadata.tsv와 w2v_tensor.tsv 파일이 생성된다.
- 해당 파일들을 다운로드한 후, [링크](https://projector.tensorflow.org/)로 이동해서 데이터를 업로드한다.

In [14]:
!python -m gensim.scripts.word2vec2tensor --input ~/aiffel/aiffel_projects/goingdeeper/GD5_word_embedding/w2v --output ~/aiffel/aiffel_projects/goingdeeper/GD5_word_embedding/w2v

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
2021-10-06 07:29:11,451 - word2vec2tensor - INFO - running /opt/conda/lib/python3.7/site-packages/gensim/scripts/word2vec2tensor.py --input /aiffel/aiffel/aiffel_projects/goingdeeper/GD5_word_embedding/w2v --output /aiffel/aiffel/aiffel_projects/goingdeeper/GD5_word_embedding/w2v
2021-10-06 07:29:11,451 - keyedvectors - INFO - loading projection weights from /aiffel/aiffel/aiffel_projects/goingdeeper/GD5_word_embedding/w2v
2021-10-06 07:29:12,436 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (10363, 100) matrix of type float32 from /aiffel/aiffel/aiffel_projects/goingdeeper/GD5_word_embedding/w2v', 'binary': False, 'en

![image](https://user-images.githubusercontent.com/80008411/136160255-a4f2e2ae-e1ca-4304-a2ab-db7548fbe8c8.png)


<hr>

## 워드 임베딩 2: FastText

- 페이스북에서 개발
- 메커니즘 자체는 Word2Vec을 그대로 따르지만, 문자 단위 n-gram(character-level n-gram) 표현을 학습한다는 점에서 다르다. 즉 Word2Vec은 단어를 더 이상 깨질 수 없는 단위로 구분하는 반면, FastText는 단어 내부의 내부 단어(subword)들을 학습한다.
- n-gram에서 n은 단어들이 얼마나 분리될지 결정하는 하이퍼파라미터
- n을 3으로 잡은 tri-gram은 단어 partial을 par, art, rti, tia, ial로 분리하고 이들을 벡터로 만든다. 정확히는 시작과 끝을 <로 구분하여 <pa, art, rti, tia, ial, al>라는 6개의 subword 토큰을 벡터로 만든다.
- 여기에 추가로 \<partial> 자체도 벡터화한다.
- 실제로는 n을 최소값, 최대값으로 범위를 설정할 수 있는데, gensim 패키지에서는 기본값으로 각각 3, 6이 설정되어 있다. 이 경우 다음과 같은 subword가 벡터화된다.
> <pa, art, rti, ita, ial, al>, <par, arti, rtia, tial, ial>, 중략, \<partial>

#### - 학습 방법

- 마찬가지로 네거티브 샘플링을 사용한다. 즉 (중심 단어, 주변 단어)의 쌍을 가지고 이 쌍이 positive인지, negative인지 예측을 진행하는 것.
- 다만 Word2Vec과의 차이는 학습 과정에서 중심 단어에 속한 문자 단위 n-gram 단어 벡터들을 모두 업데이트한다는 점이다.

#### - OOV와 오타에 대한 대응

- Word2Vec과는 달리 OOV와 오타에 robust하다. 즉 단어장에 없는 단어라도, 해당 단어의 n-gram이 다른 단어에 존재하면 이로부터 벡터값을 얻는 것이다.

In [15]:
from gensim.models import FastText

fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)

In [16]:
# Word2Vec에서 에러가 발생했던 단어들을 FastText 모델에 넣어보자
fasttext_model.wv.most_similar('overacting')

[('resolving', 0.9394947290420532),
 ('fluctuating', 0.938098132610321),
 ('malting', 0.9359013438224792),
 ('mounting', 0.9339149594306946),
 ('emptying', 0.9323922991752625),
 ('extracting', 0.9315359592437744),
 ('shooting', 0.9312771558761597),
 ('debilitating', 0.929542601108551),
 ('overwhelming', 0.9291026592254639),
 ('lifting', 0.9286652207374573)]

In [17]:
fasttext_model.wv.most_similar('memoryy')

[('memory', 0.9469051361083984),
 ('musical', 0.8639096021652222),
 ('mechanisms', 0.8625136613845825),
 ('mechanism', 0.8617560267448425),
 ('basic', 0.8538733124732971),
 ('imagine', 0.8524118661880493),
 ('mechanical', 0.8492914438247681),
 ('technical', 0.8422043323516846),
 ('intelligence', 0.8375174403190613),
 ('specific', 0.8353775143623352)]

#### - 한국어에서의 FastText

- 한국어도 FastText 방식으로 학습이 가능하다. n-gram이 음절 또는 자소 단위라고 보는 것이다.
- 1) 음절 단위
    - n=3일 때
    > 텐서플로우 = <텐서, 텐서플, 서플로, 플로우, 로우>, <텐서플로우>
- 2) 자소 단위
    - 단어의 초성, 중성, 종성을 분리한다고 하고, 종성이 존재하지 않을 경우에는 \_토큰을 대신 사용한다.
    > 텐서플로우 = <ㅌㅔ, ㅌㅔㄴ, ㅔㄴㅅ, ㄴㅅㅓ, ㅅㅓ_, 중략 >
    - 이 경우 고유명사의 의미적 특성을 학습하는 데는 크게 도움이 되지 않거나 오히려 성능이 떨어진다. 어휘를 분해하여 이해할 수 있는 복합명사는 의미적 특징을 효과적으로 잡아낼 수 있지만, 고유명사는 어휘를 분해해도 아무 이득을 볼 수 없기 때문이다.

<hr>

## 워드 임베딩 3: GloVe

- Global Vectors for Word Representation은 2014년 미국 스탠포드 대학에서 개발한 워드 임베딩 방법로닝다.
- 가장 큰 특징은 카운트 기반과 예측 기반의 방법을 모두 사용했다는 것이다.

### 잠재 의미 분석(LSA, Latent Semantic Analysis)

- 단어의 빈도를 수치화한 DTM을 차원 축소하여 밀집 표현으로 임베딩하는 방법
- 즉 DTM에 특잇값 분해를 사용해 잠재된 의미를 이끌어내는 방법론
- 결국 단어를 카운트해서 만든 DTM을 입력으로 하므로 카운트 기반의 임베딩 방법이라고 볼 수 있는데, 이는 몇 가지 한계를 가진다.
    - 1) 차원 축소의 특성으로 인해 새로운 단어가 추가되면 다시 DTM을 만들어 새로 차원 축소를 해야 한다.
    - 2) 단어 벡터간 유사도를 계산하는 측면에서 Word2Vec보다 성능이 떨어진다.

- LSA와 대조되는 예측 기반 방법은 Word2Vec과 같은 방법을 말한다. Word2Vec은 LSA보다 단어 벡터 간 유사도를 구하는 능력은 뛰어나지만, 코퍼스의 전체적인 통계 정보를 활용하지 못한다는 한계가 있다.
- 따라서 두 가지를 모두 사용하는 GloVe가 등장하게 된다.

### 윈도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix)

- GloVe를 이해하기 위해서는 윈도우 기반 동시 등장 행렬의 정의를 이해해야 한다.
- 예를 들어 다음 문장의 코퍼스가 있다고 해보자.
    - I like deep learning.
    - I like NLP.
    - I enjoy flying.
- 이로부터 만들어진 동시 등장 행렬은 다음과 같다.

![image](https://user-images.githubusercontent.com/80008411/136167412-0e3d9bcf-b822-4cdb-a7ae-48e5421d4109.png)

- 행과 열을 전체 단어장의 단어들로 구성하고, 어떤 i 단어의 윈도우 크기 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬이다.

### 동시 등장 확률(Co-occurrence Probability)

- $P(k|i)$ 동시 등장 확률은 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률이다.
- i를 중심 단어, k를 주변 단어라고 한다.

![image](https://user-images.githubusercontent.com/80008411/136167879-89754384-b358-4c79-9c1d-0118e4a48a79.png)


### 손실 함수

- GloVe는 동시 등장 행렬로부터 계산된 동시 등장 확률을 이용해 손실 함수를 설계한다.
- GloVe의 아이디어를 한 줄로 요약하면, 중심 단어 벡터와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 빈도의 로그값이 되도록 만드는 것.
- 즉 전체 코퍼스에서의 동시 등장 빈도의 로그값과 중심 단어 벡터와 주변 단어 벡터의 내적값의 차이가 최소화되도록 두 벡터의 값을 학습하는 것

- GloVe의 손실 함수 식과 변수들

![image](https://user-images.githubusercontent.com/80008411/136169006-bceb1aa6-9ca5-4c6a-b76b-8030dc031115.png)

![image](https://user-images.githubusercontent.com/80008411/136168911-cdd0c174-f68e-4b09-9922-c815ffc48e8e.png)

- $f(X_{ik})$는 동시 등장 빈도의 값으로 굉장히 낮을 경우에는 거의 도움이 되지 않는 정보이다. 그래서 여기에 가중치를 주기 위해 가중치 함수를 도입했다.
- 아래와 같이 $f(X_{ik})$ 값이 작으면 함수 값은 작아지고, 값이 크면 함수 값도 커진다. 하지만 큰 값에 대해서 지나친 가중치를 주지 않기 위해 최대값을 1로 정했다. 이는 'it is'와 같은 불용어의 동시 등장 빈도수가 높을 때 지나친 가중을 주지 않기 위함이다.

![image](https://user-images.githubusercontent.com/80008411/136169043-c1b7fb14-3fe9-424a-963d-d72b3b6ffa19.png)


### GloVe 실습

In [25]:
import nltk

nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /aiffel/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
from nltk.corpus import movie_reviews

corpus = movie_reviews.sents()

In [27]:
from glove import Corpus, Glove

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
emb = Corpus()
emb.fit(corpus, window=5)

# 벡터의 차원은 100, 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(emb.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(emb.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [28]:
model_result1 = glove.most_similar("man")
model_result2 = glove.most_similar("fiction")

print("model_result1", model_result1)
print("model_result2", model_result2)

model_result1 [('woman', 0.9541682319723492), ('young', 0.8944593575706618), ('girl', 0.8887817885942938), ('boy', 0.8826109016093757)]
model_result2 [('science', 0.9826217148867289), ('pulp', 0.9679211348516671), ('kong', 0.6976340973632392), ('hong', 0.6809053655622014)]
